<a href="https://colab.research.google.com/github/CalculatedContent/ww-phys_theory/blob/master/Full_Test_Jacobians_Resnets_SVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook for generating Full Jacobian SVD over the test data for ResNet data


June 5 2020

```
resnet20_cifar10
resnet56_cifar10
resnet110_cifar10
resnet164bn_cifar10
resnet272bn_cifar10
resnet542bn_cifar10
resnet1001_cifar10
resnet1202_cifar10
```




In [0]:
Jdir =  '/content/drive/My Drive/J_resnets_fullsvd'

thismodel = 'resnet20_cifar10'
inum_start = 0
batch_size = 100

num_classes= 10
device = 'cuda:0'
num_data = 10000


In [2]:
!pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [3]:
!pip install ipython-autotime
%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=a4b04802e1329ce1ee619d9af8ac97aa916e0f608ae73234fb2826d0e4573701
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [4]:
!pip install pytorchcv
!pip install powerlaw

     |████████████████████████████████| 440kB 2.6MB/s 
  Created wheel for powerlaw: filename=powerlaw-1.4.6-cp36-none-any.whl size=24787 sha256=ac72af737a98f10861a6aa719011a9cee010717294b6c0a53e304534aebf5801
  Stored in directory: /root/.cache/pip/wheels/e0/27/02/08d0e2865072bfd8d7c655e290521e3feca3fc22f1ac460601
Successfully built powerlaw
time: 8.09 s


In [5]:
!pip install GPUtil
import GPUtil


GPUtil.showUtilization()

  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=4431bd81b11e81f2021c2af2d479baf2e20287681ab6ab42d291897195d39367
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
time: 4.81 s


In [6]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
time: 19.1 s


In [7]:
!mkdir "$Jdir"
!ls "$Jdir"

mkdir: cannot create directory ‘/content/drive/My Drive/J_resnets_fullsvd’: File exists
time: 4.9 s


In [8]:
!cd "$Jdir";pwd

/content/drive/My Drive/J_resnets_fullsvd
time: 2.15 s


In [9]:
!import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

import math
import copy




/bin/bash: import: command not found
time: 5.38 s


### Run InstallPhysTheory notebook 

In [10]:
import sys
sys.path.append("/content/drive/My Drive/Jacobian")
import jacobian as jac
import utils

time: 1.67 s


In [11]:
def jacobian_batched_full(model, data_loader, batch_size, num_classes=10, device='cuda:0', data_dim=3*32*32): 
  '''compute a full of J^{T}J Jacobian in batch mode'''

  model.eval() 
  model = model.to(device)

  left_data_loader = data_loader 
  right_data_loader = copy.deepcopy(data_loader)

  JJMat = None 
  for left_batch, data in enumerate(left_data_loader): 
    print(left_batch) 
    features, _ = data 
    inputs = features.to(device)
    inputs.requires_grad=True
    outputs = model(inputs)
    J = jac.batch_diagJ(inputs, outputs)
    print("J shape" , J.shape, num_classes*data_dim)

    J = J.reshape(batch_size,num_classes*data_dim)
    print("J shape" , J.shape)

    JJBlock = None
    for right_batch, data in enumerate(right_data_loader):
      print(left_batch, right_batch)
      features, _ = data 
      inputs = features.to(device)
      inputs.requires_grad=True
      outputs = model(inputs)
    
      Jt = jac.batch_diagJ(inputs, outputs)
      Jt = Jt.reshape(batch_size,num_classes*data_dim)
      Jt = Jt.transpose(0,1)
      block = torch.mm(J,Jt)
    
      del Jt
      torch.cuda.empty_cache()
    
      # form square subblock
      if JJBlock is None:
          JJBlock = block
      else:
        JJBlock = torch.cat([JJBlock, block])
    
      print("JJBlock shape", right_batch, JJBlock.shape)
    
    # form rectangulae subblock
    if JJMat is None:
      print("Jmat is none")
      JJMat = JJBlock
    else:
      print("Jmat being extended")
      JJMat = torch.cat([JJMat, JJBlock],1)
    
    del J
    torch.cuda.empty_cache()
    
    print("JJMat shape", left_batch, JJMat.shape)
    return JJMat


time: 33.4 ms


In [12]:
def jacobian_full(model, data_loader, batch_size, num_classes=10, device='cuda:0', data_dim=33232): 
  '''compute a full of J^{T}J Jacobian,'''

  model.eval()
  model = model.to(device)

  J_right = jac.construct_diagJ(model, test_loader, batch_size=batch_size, device=device) 
  J_left = J_right.transpose(1,0)
  JJ = torch.mm(J_right, J_left)

  del J_right, J_left
  torch.cuda.empty_cache()

  return JJ

time: 4.82 ms


### ResNet 

In [13]:
import pytorchcv
from pytorchcv.model_provider import get_model as ptcv_get_model

for modelname in pytorchcv.model_provider._models.keys():
    if modelname.startswith('resnet') and modelname.endswith('cifar10'):
        print(modelname)

resnet20_cifar10
resnet56_cifar10
resnet110_cifar10
resnet164bn_cifar10
resnet272bn_cifar10
resnet542bn_cifar10
resnet1001_cifar10
resnet1202_cifar10
time: 88.4 ms


### Compute JJ

```
def get_data(
		batch_size=100, 
		train_range=None, 
		test_range=None, 
		random_labels=False, 
		seed = 0):
```



In [14]:
train_dataset, test_dataset, train_loader, test_loader = utils.get_data(batch_size=batch_size)
for modelname in pytorchcv.model_provider._models.keys():
    if modelname.startswith('resnet') and modelname.endswith('cifar10'):
      if modelname=='resnet542bn_cifar10':
        model = ptcv_get_model(modelname, pretrained=True)
        model.eval()
        model.to('cuda:0')

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified
time: 30.4 s


In [15]:
import powerlaw

time: 2.06 ms


In [16]:
alphas = {}
for modelname in pytorchcv.model_provider._models.keys():
    if modelname.startswith('resnet') and modelname.endswith('cifar10'):
      if modelname=='resnet542bn_cifar10':
        
        model = ptcv_get_model(modelname, pretrained=True)
        
        JJMat = jacobian_batched_full(model, test_loader, batch_size)
        
        evals = torch.eig(JJMat)
        norm=10*3*32*32
        evals = (1.0/norm)*(evals.eigenvalues.cpu().numpy()[:,0])

        fit = powerlaw.Fit(evals)
        alpha = fit.alpha
        D = fit.D

        plt.hist(evals, bins=100)
        plt.title("{}  alpha = {:0.2f} D = {:0.2f} ".format(modelname, alpha, D))
        plt.show()

        alphas[modelname]=alpha

0
J shape torch.Size([100, 10, 3, 32, 32]) 30720
J shape torch.Size([100, 30720])
0 0


RuntimeError: ignored

time: 5.94 s


In [0]:
10*3*32*32